<a href="https://colab.research.google.com/github/imAryanSingh/Medicine-Recommendation-System/blob/main/MedicineRecommendationSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [331]:
#Importing Useful Packages
import numpy as np
import pandas as pd
import nltk
from nltk.stem.porter import PorterStemmer as ps
from sklearn.feature_extraction.text import CountVectorizer
import pickle as pkl
from sklearn.metrics.pairwise import cosine_similarity as cs

In [332]:
#Importing Data Set
dataset = pd.read_csv('/content/sample_data/MedicineRecmdProj/medicine.csv')
print(dataset)

      index                                          Drug_Name Reason  \
0         1               A CN Gel(Topical) 20gmA CN Soap 75gm   Acne   
1         2  A Ret 0.05% Gel 20gmA Ret 0.1% Gel 20gmA Ret 0...   Acne   
2         3                             ACGEL CL NANO Gel 15gm   Acne   
3         4                                ACGEL NANO Gel 15gm   Acne   
4         5                              Acleen 1% Lotion 25ml   Acne   
...     ...                                                ...    ...   
9715   9716                                T Muce Ointment 5gm  Wound   
9716   9717  Wokadine 10% Solution 100mlWokadine Solution 5...  Wound   
9717   9718                               Wokadine M Onit 10gm  Wound   
9718   9719                           Wound Fix Solution 100ml  Wound   
9719   9720                              Wounsol Ointment 15gm  Wound   

                                            Description  
0                         Mild to moderate acne (spots)  
1     A

In [333]:
#The brief information about dataset is Given Below
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9720 entries, 0 to 9719
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   index        9720 non-null   int64 
 1   Drug_Name    9720 non-null   object
 2   Reason       9720 non-null   object
 3   Description  9720 non-null   object
dtypes: int64(1), object(3)
memory usage: 303.9+ KB


In [334]:
#View the first few rows of the dataset
dataset.head()
#ClicK On "SUGGESTED CHARTS" It Show More Visual Information

,index,Drug_Name,Reason,Description
0,1,A CN Gel(Topical) 20gmA CN Soap 75gm,Acne,Mild to moderate acne (spots)
1,2,A Ret 0.05% Gel 20gmA Ret 0.1% Gel 20gmA Ret 0...,Acne,A RET 0.025% is a prescription medicine that i...
2,3,ACGEL CL NANO Gel 15gm,Acne,It is used to treat acne vulgaris in people 12...
3,4,ACGEL NANO Gel 15gm,Acne,It is used to treat acne vulgaris in people 12...
4,5,Acleen 1% Lotion 25ml,Acne,treat the most severe form of acne (nodular ac...


In [335]:
#There are Total 50 (0 to 9720) Entries,8 (0 to 4) Columns.
dataset.shape

(9720, 4)

In [336]:
#There are Total 38880 Elements
dataset.size

38880

In [337]:
#Total Unique Values in every column are Shown here
dataset.nunique()

index          9720
Drug_Name      9626
Reason           50
Description     290
dtype: int64

In [338]:
#Basic Description About Dataset like count,mean,min,max,etc.
dataset.describe()
#ClicK On "SUGGESTED CHARTS" It Show More Visual Information

,index
count,9720.000000
mean,4860.500000
std,2806.066642
min,1.000000
25%,2430.750000
50%,4860.500000
75%,7290.250000
max,9720.000000


In [339]:
#dropping rows & columns containing missing values,inplace parameter set to True,means changes applied directly to DataFrame.
dataset.dropna(inplace=True)

In [340]:
#Checking count of duplicated values using sum()
dataset.duplicated().sum()

0

In [341]:
#Description Column of Dataset
dataset['Description']

0                           Mild to moderate acne (spots)
1       A RET 0.025% is a prescription medicine that i...
2       It is used to treat acne vulgaris in people 12...
3       It is used to treat acne vulgaris in people 12...
4       treat the most severe form of acne (nodular ac...
                              ...                        
9715                              used for treating warts
9716                        used to soften the skin cells
9717                                       used for scars
9718                                      used for wounds
9719    used to treat and remove raised warts (usually...
Name: Description, Length: 9720, dtype: object

In [342]:
#apply() Panda apply func to each elemnt of col.lambda x:x.split() func splits each str in col into list of words.
dataset['Reason'] = dataset['Reason'].apply(lambda x:x.split())
dataset['Description'] = dataset['Description'].apply(lambda x:x.split())
print(dataset.Reason)
print(dataset.Description)

0        [Acne]
1        [Acne]
2        [Acne]
3        [Acne]
4        [Acne]
         ...   
9715    [Wound]
9716    [Wound]
9717    [Wound]
9718    [Wound]
9719    [Wound]
Name: Reason, Length: 9720, dtype: object
0                     [Mild, to, moderate, acne, (spots)]
1       [A, RET, 0.025%, is, a, prescription, medicine...
2       [It, is, used, to, treat, acne, vulgaris, in, ...
3       [It, is, used, to, treat, acne, vulgaris, in, ...
4       [treat, the, most, severe, form, of, acne, (no...
                              ...                        
9715                         [used, for, treating, warts]
9716                 [used, to, soften, the, skin, cells]
9717                                   [used, for, scars]
9718                                  [used, for, wounds]
9719    [used, to, treat, and, remove, raised, warts, ...
Name: Description, Length: 9720, dtype: object


In [343]:
#takes a string as argument and returns same string, all spaces " " replaced with an empty "" string
dataset['Description'] = dataset['Description'].apply(lambda x:[i.replace(" ","") for i in x])

In [344]:
#Combning Description,Reason column as single column Disease,So that main cause highlight
dataset['Disease'] = dataset['Description'] + dataset['Reason']

In [345]:
#We reduced the data into important data like index,Drug_name,Disease
new_data = dataset[['index','Drug_Name','Disease']]
print(new_data)

      index                                          Drug_Name  \
0         1               A CN Gel(Topical) 20gmA CN Soap 75gm   
1         2  A Ret 0.05% Gel 20gmA Ret 0.1% Gel 20gmA Ret 0...   
2         3                             ACGEL CL NANO Gel 15gm   
3         4                                ACGEL NANO Gel 15gm   
4         5                              Acleen 1% Lotion 25ml   
...     ...                                                ...   
9715   9716                                T Muce Ointment 5gm   
9716   9717  Wokadine 10% Solution 100mlWokadine Solution 5...   
9717   9718                               Wokadine M Onit 10gm   
9718   9719                           Wound Fix Solution 100ml   
9719   9720                              Wounsol Ointment 15gm   

                                                Disease  
0             [Mild, to, moderate, acne, (spots), Acne]  
1     [A, RET, 0.025%, is, a, prescription, medicine...  
2     [It, is, used, to, treat, a

In [346]:
# lambda func takes list of str as argument and returns single str that is concatenation of all strings the list
new_data['Disease'] = new_data['Disease'].apply(lambda x:" ".join(x))
print(new_data)

      index                                          Drug_Name  \
0         1               A CN Gel(Topical) 20gmA CN Soap 75gm   
1         2  A Ret 0.05% Gel 20gmA Ret 0.1% Gel 20gmA Ret 0...   
2         3                             ACGEL CL NANO Gel 15gm   
3         4                                ACGEL NANO Gel 15gm   
4         5                              Acleen 1% Lotion 25ml   
...     ...                                                ...   
9715   9716                                T Muce Ointment 5gm   
9716   9717  Wokadine 10% Solution 100mlWokadine Solution 5...   
9717   9718                               Wokadine M Onit 10gm   
9718   9719                           Wound Fix Solution 100ml   
9719   9720                              Wounsol Ointment 15gm   

                                                Disease  
0                    Mild to moderate acne (spots) Acne  
1     A RET 0.025% is a prescription medicine that i...  
2     It is used to treat acne vu

<ipython-input-346-1dfa13e15553>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['Disease'] = new_data['Disease'].apply(lambda x:" ".join(x))


In [347]:
#takes a string as its argument and returns the same string in lowercase.
new_data['Disease'] = new_data['Disease'].apply(lambda x:x.lower())
print(new_data)

      index                                          Drug_Name  \
0         1               A CN Gel(Topical) 20gmA CN Soap 75gm   
1         2  A Ret 0.05% Gel 20gmA Ret 0.1% Gel 20gmA Ret 0...   
2         3                             ACGEL CL NANO Gel 15gm   
3         4                                ACGEL NANO Gel 15gm   
4         5                              Acleen 1% Lotion 25ml   
...     ...                                                ...   
9715   9716                                T Muce Ointment 5gm   
9716   9717  Wokadine 10% Solution 100mlWokadine Solution 5...   
9717   9718                               Wokadine M Onit 10gm   
9718   9719                           Wound Fix Solution 100ml   
9719   9720                              Wounsol Ointment 15gm   

                                                Disease  
0                    mild to moderate acne (spots) acne  
1     a ret 0.025% is a prescription medicine that i...  
2     it is used to treat acne vu

<ipython-input-347-b1a8ffeb3f9c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['Disease'] = new_data['Disease'].apply(lambda x:x.lower())


In [348]:
ps = PorterStemmer()

In [349]:
"""
CountVectorizer class used to convert text documents into vectors of counts.
stop_words parameter specifies list of words that shuld be ignored when creating the vector.
max_features parameter specifies max number of features tht shuld be used.
set to english means common English words like "the,and,is,etc" will be ignored.
set to 5000 means tht only 5000 most common words will be used to create the vector.
"""
cv = CountVectorizer(stop_words='english',max_features=5000)

In [350]:
"""
takes str of txt as input and returns a stemmed version of the text
The stemming process involves removing suffix like "ing,ly,s,etc" to reduce words to their root form.
useful for information retrieval and txt mining, as it help to imprve accuracy of result
func joins the stemmed words back together into single str and returns it.
"""
def stem(text):
  y = []

  for i in text.split():
    y.append(ps.stem(i))

  return " ".join(y)

In [351]:
#Now we applied stem to each element of Disease
new_data['Disease'] = new_data['Disease'].apply(stem)

<ipython-input-351-8bd5caef4e8d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['Disease'] = new_data['Disease'].apply(stem)


In [352]:
#no of rows is the no of documents in the data set and the no of columns is the no of features in the vector representation.
vectors = cv.fit_transform(new_data['Disease']).toarray()
print(vectors.shape)

(9720, 806)


In [353]:
#list of features that used to create vector representation,list of 5000 most common word
cv.get_feature_names_out()

array(['025', '12', '16', '18', 'abdomin', 'abl', 'ach', 'acid', 'acn',
       'acne', 'acquir', 'action', 'activ', 'acut', 'acute', 'adequ',
       'adhd', 'adjunct', 'adolesc', 'adult', 'adults', 'affect', 'ag',
       'age', 'aids', 'allerg', 'allergen', 'allergi', 'allow', 'alon',
       'alzheim', 'alzheimer', 'alzheimerâ', 'amoebiasi', 'anaemia',
       'anal', 'angina', 'angl', 'ani', 'ankylos', 'anorexia', 'anoth',
       'anti', 'antioxid', 'antipsychot', 'antiretrovir', 'anxieti',
       'anxiou', 'anxious', 'apnoea', 'appear', 'appetit', 'appetite',
       'appli', 'appropri', 'area', 'arrhythmia', 'arrhythmiasi',
       'arteri', 'arthralgia', 'arthriti', 'associ', 'atherothrombot',
       'athleteâ', 'atop', 'atrial', 'attack', 'awak', 'b1', 'b2', 'b3',
       'b5', 'b6', 'babi', 'backache', 'bacteri', 'bacteria', 'balanc',
       'balanitis', 'bandag', 'becom', 'behaviour', 'beliefs', 'benefit',
       'beta', 'biliari', 'biotin', 'bite', 'blackhead', 'blackheads',
      

In [354]:
#Check cosine value of two argumnt vector,used to measure the similarity between text documents.
cs(vectors)

array([[1.        , 0.25197632, 0.43643578, ..., 0.        , 0.        ,
        0.        ],
       [0.25197632, 1.        , 0.25660012, ..., 0.19245009, 0.1490712 ,
        0.0860663 ],
       [0.43643578, 0.25660012, 1.        , ..., 0.11111111, 0.0860663 ,
        0.0993808 ],
       ...,
       [0.        , 0.19245009, 0.11111111, ..., 1.        , 0.77459667,
        0.2981424 ],
       [0.        , 0.1490712 , 0.0860663 , ..., 0.77459667, 1.        ,
        0.34641016],
       [0.        , 0.0860663 , 0.0993808 , ..., 0.2981424 , 0.34641016,
        1.        ]])

In [355]:
"""
Similar var is a NumPy array contain cosine similarity b/w each pair of vectors in vectors array.
The cosine similarity b/w two vectors is b/w 0 & 1,0 means vectors are completely diffrnt & 1 means vectors r exactly same.
"""
similar = cs(vectors)
print(similar[1])

[0.25197632 1.         0.25660012 ... 0.19245009 0.1490712  0.0860663 ]


In [359]:
"""
recommend() function takes medicine parameter & return list of similar medicines.
func first finds row index of medicine in new_data using isin() method then,
use index attribute of resulting data frame to get row index of the first row.
func then use distance arry to get distances between medicine & other medicines in data frame.
func then sorts distance in descending order & selects first five medicine,then prints names of five medicines.
"""
def recommend(medicine):
    medicine_index = new_data[new_data['Drug_Name'] == medicine].index[0]
    distances = similar[medicine_index]
    medicines_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]

    for i in medicines_list:
        print(new_data.iloc[i[0]].Drug_Name)

In [361]:
recommend("Danil Antidandruff Shampoo 100ml")

Angiografin 65% Infusion 50ml
Blise Face Wash Gel 100gm
Chupp LS Syrup 100ml
Cutirinz Foaming Face Wash 100ml
Danil Antidandruff Shampoo 100ml
